In [4]:
import sys
import yaml 
from pathloss_3gpp_eq7 import *

In [3]:
def get_from_dict(key, data, res=None, curr_level = 1, min_level = 1):
    """Fonction qui retourne la valeur de n'importe quel clé du dictionnaire
       key: clé associé à la valeur recherchée
       data: dictionnaire dans lequel il faut chercher
       les autres sont des paramètres par défaut qu'il ne faut pas toucher"""
    if res:
        return res
    else:
        # data IS a dictionary
        for k, v in data.items():
            if k == key and curr_level >= min_level:
                #print(f"return data[k] = {data[k]} k = {k}")
                return data[k]
            if type(v) is dict:
                level = curr_level + 1
                res = get_from_dict(key, v, res, level, min_level)
    return res 

In [3]:
# Fonction verifiant la présence d'un string dans un fichier YAML
# Retourne True si le string est présent et False sinon
def check_string_presence_in_yaml(string_to_check, yaml_data):
    for device_type, devices in yaml_data.items():
        for device_name, device_info in devices.items():
            if string_to_check == device_name:
                return True
    return False

def ERROR(msg , code = 1):
    print("\n\n\nERROR\nPROGRAM STOPPED!!!\n")
    if msg:
        print(msg)
    print(f"\n\texit code = {code}\n\n\t\n")
    sys.exit(code)

In [20]:
import sys
import yaml 
fichier = "lab2_eq7_cas.yaml"
with open(fichier,"r" ) as file:
    data= yaml.load(file, Loader=yaml. FullLoader)

    
# next(iter(data))

In [24]:
string_extrait = get_from_dict('read', get_from_dict('VISIBILITY', data))
string_extrait = get_from_dict('scenario', data)
import re

def check_string_format(string):
    if string[0].isupper() and string[1].isupper() and string[-1].islower():
        print("The given string format is valid.")
    else:
        warning_message = 'Warning: The string "{}" does not match the expected format (first two characters should be uppercase, last character lowercase).\n'
        print(warning_message.format(string))

# Example usage
scenario = get_from_dict('scenario', data)
check_string_format(scenario)


In [ ]:
rma_nlos()

In [12]:
s = """WARNING : la distance entre l'UE 14 et l'antenne 8 est plus grande que 5 km.
Nous considerons un pathloss valant INFINI entre ces deux equipements
WARNING : la distance entre l'UE 14 et l'antenne 8 est plus grande que 5 km.
Nous considerons un pathloss valant INFINI entre ces deux equipements
"""
count = s.count("WARNING")

print(s)
print(count)

Nous considerons un pathloss valant INFINI entre ces deux equipements
Nous considerons un pathloss valant INFINI entre ces deux equipements

2


In [70]:
def check_coord_files_mode(data):
    nom_du_fichier = ""
    
    coord_files_mode = get_from_dict("COORD_FILES", data)
    if coord_files_mode:
        if 'read' in coord_files_mode.keys() and 'write' not in coord_files_mode.keys():
            mode = True
            nom_du_fichier = get_from_dict("read", data)
            return nom_du_fichier, mode
            
        elif 'write' in coord_files_mode.keys() and 'read' not in coord_files_mode.keys():
            mode = False
            nom_du_fichier = get_from_dict("write", data)
            return nom_du_fichier, mode
            
    else:
        ERROR("La clé COORD_FILES n'est pas définie.")
    

In [71]:
check_coord_files_mode(data)

('coords_lab2_eq7.txt', True)

In [21]:
x= etude_path_loss["PATHLOSS"]
x

{'model': 'okumura', 'scenario': 'urban_large'}

In [12]:
antenna1= x["Antenna1"]

In [13]:
antenna1

{'number': 25}

In [14]:
n = antenna1["number"]


In [57]:
def sanity_check_coordinates_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

        id_antenna = -1
        id_ue = -1

        for line in lines:
            # Vérifier s'il y a des lignes vides
            if line.strip() == '':
                ERROR(f"Le fichier '{filename}' contient des lignes vides.")

            # Vérifier le nombre de colonnes
            parts = line.strip().split()
            if parts[0] == 'antenna':
                # Vérifier le format des lignes antenna
                if len(parts) != 5:
                    ERROR(f"Le format de la ligne antenna dans le fichier '{filename}' est incorrect. Il doit y avoir 5 colonnes : string chiffre string chiffre chiffre")
                # Vérifier si l'identifiant est incrémenté correctement
                current_id = int(parts[1])
                if current_id != id_antenna + 1:
                    ERROR(f"L'identifiant de l'antenne dans le fichier '{filename}' n'est pas incrémenté correctement: {line.strip()}")
                id_antenna = current_id
            elif parts[0] == 'ue':
                # Vérifier le format des lignes ue
                if len(parts) != 6:
                    ERROR(f"Le format de la ligne ue dans le fichier '{filename}' est incorrect. Il doit y avoir 6 colonnes : string chiffre string chiffre chiffre string")
                # Vérifier si l'identifiant est incrémenté correctement
                current_id = int(parts[1])
                if current_id != id_ue + 1:
                    ERROR(f"""L'identifiant de l'UE dans le fichier '{filename}' n'est pas incrémenté correctement: {line.strip()}. 
                    Se rappeler que l'id commence a 0 et doit s'incrementer un a un par la suite.""")
                id_ue = current_id
            else:
                ERROR(f"La première colonne de la ligne n'est ni 'antenna' ni 'ue': {line.strip()} dans le fichier '{filename}'.")

# Exemple d'utilisation :
filename = 'coords_lab2_eq7.txt'
sanity_check_coordinates_file(filename)
print("Les sanity checks ont été passés avec succès.")





ERROR
PROGRAM STOPPED!!!

L'identifiant de l'antenne dans le fichier 'coords_lab2_eq7.txt' n'est pas incrémenté correctement: antenna	24	Antenna4	3.5	5.999999999999999

	exit code = 1

	



SystemExit: 1

In [91]:
import os
def sanity_check_visibility_file(filename, nombre_ue):
    # Vérifier si le fichier existe 
    if not os.path.exists(filename):
        ERROR(f"Le fichier {filename} n'existe pas dans le repertoire courant.")
    
    with open(filename, 'r') as file:
        lines = file.readlines()

        # Vérifier s'il y a des lignes vides
        if any(line.strip() == '' for line in lines):
            ERROR(f"Le fichier '{filename}' contient des lignes vides.")

        # Vérifier le bon nombre de colonnes et l'absence de répétitions de chiffres
        first_digits_set = set()
        for line in lines:
            ue_numbers = line.strip().split()
            if len(ue_numbers) != len(set(ue_numbers)):
                ERROR(f"Il y a des répétitions de id d'antenne dans une ligne du fichier '{filename}'.")

            if len(ue_numbers) < 2:
                ERROR(f"Chaque ligne du fichier '{filename}' doit contenir au moins deux chiffres (UE et antenne).")

            # Vérifier si le premier chiffre est différent des précédents
            first_digit = ue_numbers[0]
            if first_digit in first_digits_set:
                ERROR(f"""Le premier chiffre de la ligne du fichier '{filename}' est en double : {first_digit}.
                Le premier chiffre d'une ligne represente une UE et ne peut donc pas se retrouver sur une autre ligne.""")
            first_digits_set.add(first_digit)

        # Vérifier le nombre de lignes
        ue_count = len(lines)
        min_ue_count = 0.05 * nombre_ue
        max_ue_count = 0.30 * nombre_ue
        if ue_count < min_ue_count or ue_count > max_ue_count:
            ERROR(f"""Le nombre de lignes ({ue_count}) dans le fichier '{filename}' n'est pas compris entre 5% et 30% du nombre d'UE spécifié ({nombre_ue}).
            SVP, mettre un nombre de ligne entre {min_ue_count} et {max_ue_count} dans le fichier '{filename}'.""")



# Exemple d'utilisation :
filename = 'visibdfdility_lab2_eq7.txt'
nombre_ue = 20  # Remplacer par le nombre réel d'UE
result = sanity_check_visibility_file(filename, nombre_ue)
print("Les sanity checks ont été passés avec succès." if result else "Il y a des erreurs dans le fichier de visibilité.")





ERROR
PROGRAM STOPPED!!!

Le fichier visibdfdility_lab2_eq7.txt n'existe pas dans le repertoire courant.

	exit code = 1

	



SystemExit: 1

In [25]:
etude_path_loss1 = get_from_dict(key="number", data=data)
etude_path_loss2 = get_from_dict(key="number", data=data)
print(etude_path_loss1,etude_path_loss2)

25 25


In [1]:
import sys
import math
import yaml
import random
import os
import argparse
import matplotlib.pyplot as plt

class Antenna:

     def __init__(self, id):
        self.id = id          #id de l'antenne (int)
        self.frequency = None # Antenna frequency in GHz
        self.height = None    # Antenna height
        self.group = None     # group défini dans la base de données (str)
        self.coords = None    # tuple contenant les coordonnées (x,y) 
        self.assoc_ues = []   # liste avec les id des UEs associés à l'antenne
        self.scenario = None  # pathloss scénario tel que lu du fichier de cas (str)
        self.gen = None       # type de géneration de coordonnées: 'g', 'a', etc. (str)

    
    
class UE:

     def __init__(self, id, app_name):
        self.id= id           # id de l'UE (int)
        self.height = None    # UE height
        self.group = None     # group défini dans la base de données (str)
        self.coords=None      # tuple contenant les coordonnées (x,y) 
        self.app=app_name     # nom de l'application qui tourne dans le UE (str)
        self.assoc_ant=None   # id de l'antenne associée à l'UE (int)
        self.los = True       # LoS ou non (bool)
        self.gen = None       # type de géneration de coordonnées: 'g', 'a', etc. (str)

class Pathloss:
     def __init__(self, id_ue, id_ant):
        self.id_ue = id_ue   # ID de l'ue
        self.id_ant = id_ant # ID de l'antenne
        self.value = None   # Valeur du pathloss


# Fonction permettant d'afficher un message d'erreur et de stopper le programme
def ERROR(msg , code = 1):
    print("\n\n\nERROR\nPROGRAM STOPPED!!!\n")
    if msg:
        print(msg)
    print(f"\n\texit code = {code}\n\n\t\n")
    sys.exit(code)

# Fonction calculant la distance entre deux point sur le terrain
def calculate_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
# Fonction donnant le group et les coords a partir du ID d'un objet dans une liste du meme objet
def get_group_and_coords_by_id(object_list, target_id):
    for object in object_list:
        if object.id == target_id:
            return object.group, object.coords
    return None  

def verify_okumura_conditions(fc,ht,hr, antenna_group, ue_group): 
    if fc > 1500 :
        ERROR(f"""La fréquence {fc} MHz du groupe d'antenne '{antenna_group}' introduite dans le fichier de cas YAML est plus grande que 1.5 GHz. 
Le model okumura ne s'applique pas. 
Veuillez changer le groupe de l'antenne consideree dans le fichier YAML de cas ou modifier l'attribut 'frequency' du groupe {antenna_group} dans le fichier device_db.yaml""")
    if fc < 150 :
        ERROR(f"""La fréquence {fc} MHz du groupe d'antenne '{antenna_group}' introduite dans le fichier de cas YAML est plus petite que 0.15 GHz.
Le model okumura ne s'applique pas. 
Veuillez changer le groupe de l'antenne consideree dans le fichier YAML de cas ou modifier l'attribut 'frequency' du groupe {antenna_group} dans le fichier device_db.yaml""")
    if ht > 300 :
        ERROR(f"""La hauteur {ht} metres du groupe d'antenne '{antenna_group}' introduite dans le fichier de cas YAML est plus grande que 300 metres. 
Le model okumura ne s'applique pas. 
Veuillez changer le groupe de l'antenne consideree dans le fichier YAML de cas ou modifier l'attribut 'height' du groupe {antenna_group} dans le fichier device_db.yaml""")
    if ht < 30 : 
        ERROR(f"""La hauteur {ht} metres du groupe d'antenne '{antenna_group}' introduite dans le fichier de cas YAML est plus petite que 30 metres. 
Le model okumura ne s'applique pas.
Veuillez changer le groupe de l'antenne consideree dans le fichier YAML de cas ou modifier l'attribut 'height' du groupe {antenna_group} dans le fichier device_db.yaml""")
    if hr > 10 :
        ERROR(f"""La hauteur {ht} metres du groupe d'UE '{ue_group}' introduite dans le fichier de cas YAML est plus grande que 10 metres. 
Le model okumura ne s'applique pas. 
Veuillez changer le groupe de l'ue consideree dans le fichier YAML de cas ou modifier l'attribut 'height' du groupe {ue_group} dans le fichier device_db.yaml""")
    if hr < 1 : 
        ERROR(f"""La hauteur {ht} metres du groupe d'UE '{ue_group}' introduite dans le fichier de cas YAML est plus petite que 1 metres. 
Le model okumura ne s'applique pas. 
Veuillez changer le groupe de l'ue consideree dans le fichier YAML de cas ou modifier l'attribut 'height' du groupe {ue_group} dans le fichier device_db.yaml""")
    return True



def okumura(fichier_de_cas, fichier_de_device, antenna_id, ue_id, antennas, ues):
    model = get_from_dict('model', fichier_de_cas)
    scenario = get_from_dict('scenario', fichier_de_cas)
    warning_message = ""
    if model == "okumura" and scenario == "urban_small":
        antenna_group, antenna_coords = get_group_and_coords_by_id(antennas, antenna_id)
        ue_group, ue_coords = get_group_and_coords_by_id(ues, ue_id)
        fc = 1000*get_from_dict('frequency', get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        ht = get_from_dict('height', get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        hr = get_from_dict('height', get_from_dict(ue_group,fichier_de_device))
        verify_okumura_conditions(fc,ht,hr, antenna_group, ue_group)        
        distance = calculate_distance(antenna_coords, ue_coords)
        
        A = (1.1 * math.log10(fc) - 0.7) * hr - 1.56 * math.log10(fc) + 0.8
            
        if distance < 1 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus petite que 1 km.
Nous considerons un pathloss valant 0 entre ces deux equipements\n"""
            pathloss = 0
        elif distance > 20 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus grande que 20 km.
Nous considerons un pathloss valant INFINI entre ces deux equipements\n"""            
            pathloss = 1000000000000000000000000000000000000000000
        else:
            pathloss = 69.55 + 26.16 * math.log10(fc) - 13.82 * math.log10(ht) - A + (44.9 - 6.55 * math.log10(ht)) * math.log10(distance)
        
        return pathloss, warning_message, distance, ht, fc, hr, scenario
    
    if model == "okumura" and scenario == "urban_large":
        antenna_group, antenna_coords = get_group_and_coords_by_id(antennas, antenna_id)
        ue_group, ue_coords = get_group_and_coords_by_id(ues, ue_id)
        fc = 1000*get_from_dict('frequency', get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        ht = get_from_dict('height', get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        hr = get_from_dict('height', get_from_dict(ue_group,fichier_de_device))
        verify_okumura_conditions(fc,ht,hr, antenna_group, ue_group)
        distance = calculate_distance(antenna_coords, ue_coords)
        
        if fc < 300:
            A = 8.29 * (math.log10(1.54 * hr))**2 - 1.1
        elif fc >= 300:
            A = 3.2 * (math.log10(11.75 * hr))**2 - 4.97
        
        if distance < 1 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus petite que 1 km.
Nous considerons un pathloss valant 0 entre ces deux equipements\n"""
            pathloss = 0
        elif distance > 20 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus grande que 20 km.
Nous considerons un pathloss valant INFINI entre ces deux equipements\n"""            
            pathloss = 1000000000000000000000000000000000000000000
        else:       
            pathloss = 69.55 + 26.16 * math.log10(fc) - 13.82 * math.log10(ht) - A + (44.9 - 6.55 * math.log10(ht)) * math.log10(distance)
        
        return pathloss, warning_message
    
    if model == "okumura" and scenario == "suburban":
        antenna_group, antenna_coords = get_group_and_coords_by_id(antennas, antenna_id)
        ue_group, ue_coords = get_group_and_coords_by_id(ues, ue_id)
        fc = 1000*get_from_dict('frequency', get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        ht = get_from_dict('height',get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        hr = get_from_dict('height', get_from_dict(ue_group,fichier_de_device))
        verify_okumura_conditions(fc,ht,hr, antenna_group, ue_group)
        distance = calculate_distance(antenna_coords, ue_coords)
        
        A = (1.1 * math.log10(fc) - 0.7) * hr - 1.56 * math.log10(fc) + 0.8

        if distance < 1 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus petite que 1 km.
Nous considerons un pathloss valant 0 entre ces deux equipements\n"""
            pathloss = 0
        elif distance > 20 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus grande que 20 km.
Nous considerons un pathloss valant INFINI entre ces deux equipements\n"""            
            pathloss = 1000000000000000000000000000000000000000000
        else:
            pathloss_urban_small = 69.55 + 26.16 * math.log10(fc) - 13.82 * math.log10(ht) - A + (44.9 - 6.55 * math.log10(ht)) * math.log10(distance)
            pathloss = pathloss_urban_small - 2 * (math.log10(fc / 28))**2 - 5.4
        
        return pathloss, warning_message, distance, ht, fc, hr, scenario
    
    if model == "okumura" and scenario == "open":
        antenna_group, antenna_coords = get_group_and_coords_by_id(antennas, antenna_id)
        ue_group, ue_coords = get_group_and_coords_by_id(ues, ue_id)
        fc = 1000*get_from_dict('frequency', get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        ht = get_from_dict('height', get_from_dict(antenna_group, get_from_dict(next(iter(fichier_de_device)), fichier_de_device)))
        hr = get_from_dict('height', get_from_dict(ue_group,fichier_de_device))
        verify_okumura_conditions(fc,ht,hr, antenna_group, ue_group)
        distance = calculate_distance(antenna_coords, ue_coords)
        
        A = (1.1 * math.log10(fc) - 0.7) * hr - 1.56 * math.log10(fc) + 0.8
        
        if distance < 1 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus petite que 1 km.
Nous considerons un pathloss valant 0 entre ces deux equipements\n"""
            pathloss = 0
        elif distance > 20 :
            warning_message = f"""WARNING : la distance entre l'UE {ue_id} et l'antenne {antenna_id} est plus grande que 20 km.
Nous considerons un pathloss valant INFINI entre ces deux equipements\n"""            
            pathloss = 1000000000000000000000000000000000000000000
        else:
            pathloss_urban_small = 69.55 + 26.16 * math.log10(fc) - 13.82 * math.log10(ht) - A + (44.9 - 6.55 * math.log10(ht)) * math.log10(distance)
            pathloss = pathloss_urban_small - 4.78 * (math.log10(fc))**2 + 18.33 * math.log10(fc) - 40.94

        return pathloss, warning_message, distance, ht, fc, hr, scenario

    # Si aucun cas n'est sélectionnee :
    # FAIRE UN MESSAGE D'ERREUR CORRESPONDANT
    ERROR("""SVP, entrer un model et un scenario dans le fichier de cas YAML parmi les propositions suivantes (model,scenario) :
           (model : okumura, scenario : urban_small)
           (model : okumura, scenario : urban_large)
           (model : okumura, scenario : suburban)
           (model : okumura, scenario : open)
          """)
    return 0



In [2]:
import yaml
# Fonction permettant de lire un fichier YAML 
def read_yaml_file(fname):
    # Fonction utilisée pour lire les fichiers de type .yaml
    # fname: nom du fichier .yaml à lire
    # le retour de la fonction est un dictionnaire avec toute l'information qui se trouve
    # dans le fichier .yaml
    # Si vous préférez vous pouvez utiliser une autre fonction pour lires les fichiers
    # de type .yaml.
    # À noter que dans cette fonction il faut ajouter les vérifications qui s'imposent
    # par exemple, l'existance du fichier
    
    # Vérifier l'existence du fichier
    if not os.path.exists(fname):
        raise FileNotFoundError(f"Le fichier {fname} n'existe pas.")

    # Ouvrir et lire le fichier YAML
    with open(fname, 'r') as file:
        return yaml.safe_load(file)


In [3]:
from pathloss_3gpp_eq7 import *
import os
fichier_de_cas = read_yaml_file("lab2_eq7_cas.yaml")
fichier_de_device = read_yaml_file("devices_db.yaml")
antennas = []
ues = []
antenna = Antenna(0)
antenna.group = "Antenna1"
antenna.coords = [83.0, 551.0]
ue = UE(0,"app1")
ue.group = "UE1-App1"
ue.coords = [2500, 1142.8571]
antennas.append(antenna)
ues.append(ue)
pathloss, warning_message = rma_los(fichier_de_cas, fichier_de_device, antenna.id, ue.id, antennas, ues)
print("Cas RMa LoS :")
print("pathloss : ",pathloss)
pathloss, warning_message = rma_nlos(fichier_de_cas, fichier_de_device, antenna.id, ue.id, antennas, ues)
print("Cas RMa NLoS :")
print(warning_message)
print("pathloss : ",pathloss)
pathloss, warning_message = uma_los(fichier_de_cas, fichier_de_device, antenna.id, ue.id, antennas, ues)
print("Cas UMa LoS :")
print(warning_message)
print("pathloss : ",pathloss)
pathloss, warning_message = uma_nlos(fichier_de_cas, fichier_de_device, antenna.id, ue.id, antennas, ues)
print("Cas UMa NLoS :")
print(warning_message)
print("pathloss : ",pathloss)
pathloss, warning_message = umi_los(fichier_de_cas, fichier_de_device, antenna.id, ue.id, antennas, ues)
print("Cas UMi LoS :")
print(warning_message)
print("pathloss : ",pathloss)
pathloss, warning_message = umi_nlos(fichier_de_cas, fichier_de_device, antenna.id, ue.id, antennas, ues)
print("Cas UMi NLoS :")
print(warning_message)
print("pathloss : ",pathloss)


Cas RMa LoS :
pathloss :  133.70410905705185
Cas RMa NLoS :

pathloss :  178.50771572276582
Cas UMa LoS :

pathloss :  134.72447166075284
Cas UMa NLoS :

pathloss :  175.19588828953368
Cas UMi LoS :

pathloss :  135.89915682039342
Cas UMi NLoS :

pathloss :  173.10059929412287


Le nombre de lignes (4) n'est pas compris entre 5% et 30% du nombre d'UE spécifié (100).
